### The RIDB API
<img src="images/hobbit_house_expedia.jpeg"style="display: inline-block">
<br>
image source: expedia.com
<br>
<br>
For an RIDB API Key: https://ridb.recreation.gov/?action=register

In [1]:
import pandas as pd 
import numpy as np
import config
import requests
import json
from pandas.io.json import json_normalize

#### Get the data from RDIB

In [ ]:
ridb_facilities_url = config.RIDB_ENDPOINT + "/facilities"
camping_params = params=dict(activity_id=9, state='OR', apiKey = config.RIDB_API_KEY,\
                             latitude=45.3300, longitude=-121.7089, radius=5)
response = requests.get(ridb_facilities_url,camping_params)

In [ ]:
mock_url = "http://" + config.LAMP_IP + "/ridb_mthood_camping_mock.json"
response = pd.read_json(mock_url)

In [ ]:
camping_json  = json.loads(response.text)
camping_df = json_normalize(camping_json['RECDATA'])

In [ ]:
camping_df.head()

In [ ]:
camping_df.LastUpdatedDate.unique()

In [ ]:
camping_df.LegacyFacilityID.unique()

#### Clean up the data

In [ ]:
camping_df = camping_df.replace('', np.nan)

Drop geojson coordinates - Lat/Long information duplicated, mysql 5.4 doesnt support geojson types (but 5.7 does!)

In [ ]:
camping_df = camping_df.drop(['GEOJSON.COORDINATES','GEOJSON.TYPE'], axis=1)

In [ ]:
sum(camping_df.FacilityAdaAccess.isnull())

#### Store the data

In [ ]:
camping_df.to_csv('test.csv', index=False)

In [ ]:
csv_test = pd.read_csv('test.csv')

In [ ]:
csv_test.head()

In [ ]:
from sqlalchemy import create_engine
connectStr = "mysql+pymysql://" + config.DB_USER + ":" + config.DB_PASS + "@" + config.DB_HOST +  "/" + config.DB_NAME
engine =create_engine(connectStr)

In [ ]:
camping_df.to_sql('test',engine,if_exists='replace')

In [ ]:
sql_test = pd.read_sql('select * from test', engine, index_col='index')

In [ ]:
sql_test.head()

In [ ]:
sql_test.LastUpdatedDate.unique()

In [ ]:
sum(sql_test.LastUpdatedDate.isnull())

In [ ]:
sql_test.dropna(subset=['LastUpdatedDate'], axis=0).shape

In [ ]:
csv_test.dropna(subset=['LastUpdatedDate'], axis=0).shape

In [ ]:
camping_df.dropna(subset=['LastUpdatedDate'], axis=0).shape